In [1]:
#!pip install pandas==1.3.5 # for comaparability woth colab
#%pip install levenshtein
%pip freeze > session_info_sunflower.txt 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os 
import pickle 
import pandas as pd
import yaml
import sys
from datetime import date

import importlib
sys.path.append('../3-data-preprocessing')
import preprocessing_utils
importlib.reload(preprocessing_utils)

import traverse_and_parse as tp
importlib.reload(tp)

<module 'traverse_and_parse' from '/usr/net/zinovyee.hub/IRTG/MLSC/Encode-the-Qode/Quantlet/2-Parse-Quantlets-Folders/traverse_and_parse.py'>

In [3]:
# VARIABLE PARAMETERS
TODAY = '20231104'

# PATHS 
PATH_TO_PARSE = '../../data/QuantLet'
FILE_TYPES = ['m', 'py', 'r', 'R', 'M', 'ipynb']
TEST_PATH = '../data/Q_test'
OUTPUT_PATH = f"../../data/preprocessed/Quantlet/{TODAY}"

# CONSTANTS
IPY_CONVERT = False

if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

In [4]:
if IPY_CONVERT:
    # q_name, folder_name, code_script, type_script, metainfo_file
    language, name = '', ''
    for i, (root, directories, files) in enumerate(os.walk(PATH_TO_PARSE)):
        
        
        for file in files:
            language = file.split('.')[-1]
            
            if (language == 'ipynb'):
                if not os.path.exists(f"{root}/{file.replace('ipynb', 'py')}"):
                    print(file)
                    os.popen(f"jupyter nbconvert --to python {root}/{file}")
            else:
                continue    

In [5]:
output = tp.traverse_folder(PATH_TO_PARSE, FILE_TYPES)
repos = tp.prepare_repos_df(output['repos'])

23it [00:01, 14.82it/s]

Could not load the metafile as yaml, because of mapping values are not allowed here
  in "../../data/QuantLet/BLEM/BLEMdataGlimpse/Metainfo.txt", line 4, column 58
Could not load the metafile as yaml, because of mapping values are not allowed here
  in "../../data/QuantLet/BLEM/BLEMpredictLASSO/Metainfo.txt", line 4, column 58


27it [00:01, 10.14it/s]

Could not load the metafile as yaml, because of mapping values are not allowed here
  in "../../data/QuantLet/BLEM/BLEMplotEnergyPreds/Metainfo.txt", line 4, column 58


32it [00:03,  6.33it/s]

...

Could not load the metafile as yaml, because of while scanning for the next token
found character '\t' that cannot start any token
  in "../../data/QuantLet/SFE_class_2016/SFERhoHedging/metainfo.txt", line 14, column 10
Could not load the metafile as yaml, because of while scanning a simple key
  in "../../data/QuantLet/SFE_class_2016/SFEbsfilm/metainfo.txt", line 7, column 1
could not find expected ':'
  in "../../data/QuantLet/SFE_class_2016/SFEbsfilm/metainfo.txt", line 9, column 1


19948it [13:03, 80.49it/s]

Could not load the metafile as yaml, because of while scanning for the next token
found character '\t' that cannot start any token
  in "../../data/QuantLet/SFE_class_2016/SFEDeltaVegaHedging/metainfo.txt", line 13, column 10
Could not load the metafile as yaml, because of while scanning for the next token
found character '\t' that cannot start any token
  in "../../data/QuantLet/SFE_class_2016/SFEVegaHedging/metainfo.txt", line 14, column 10


20186it [13:05, 25.70it/s] 

(5050, 5)
(5046, 5)
(5018, 6)


In [6]:
repos.type_script = repos.type_script.str.lower()
repos = repos[repos.type_script.isin(
            ['py', 'r', 'm', 'ipynb']
        )
    ]
repos_df = repos[~repos.folder_name.str.contains('.ipynb_checkpoints')]
repos_df.loc[repos_df.type_script== 'ipynb', 'type_script']='py'
repos_df = repos_df.reset_index(drop=True)
print(repos_df.shape)

(5018, 6)


In [7]:
NAME = f'{OUTPUT_PATH}/Quantlets_{TODAY}'
with open(f'{NAME}.pkl', 'wb') as file:
    pickle.dump(repos_df, file)